# PyMem3DG Tutorial 2 - Ensure Mesh Quality
`Cuncheng Zhu, Christopher T. Lee`

This tutorial demonstrate the complementary functionalities of PyMem3DG to ensure the quality of mesh. The tutorial does not mean to be extensive but to provide the flavor and methods that PyMem3DG uses. The extensive documentations is hosted at https://rangamanilabucsd.github.io/Mem3DG/.

To demonstrate, we set up the system to solve the remaining problem from Tutorial 1, evolving closed spherical membrane with high curvature deformation. Again, the following integration is pre-runned. Uncomment $\texttt{fe.integrate()}$ to rerun them.

In [1]:
import pymem3dg.util as dg_util
import pymem3dg as dg

icoFace, icoVertex = dg.getIcosphere(1, 3)
icoVertex = dg_util.sphericalHarmonicsPerturbation(icoVertex, 5, 6, 0.1)
p = dg.Parameters()
p.bending.Kbc = 8.22e-5
p.tension.Ksg = 0.1
p.tension.At = 12.4866
p.osmotic.isPreferredVolume = True
p.osmotic.Kv = 0.02
p.osmotic.Vt = 0.7 * 3.14 * 4 / 3


c:\users\zhucu\dev\mem3dg\python_src\pymem3dg\util.py:44: RuntimeWarning: divide by zero encountered in true_divide
  phi = np.arctan(z/r) + np.pi / 2


The setup is exactly the same as the last example in tutorial 1. Without any mesh curation, the mesh will be ill-formed.

## Mesh regularization
The first strategy is to regularize the mesh, restricting the tangential movement of vertices. Regularization does not change the mesh connectivity and the number of elements in the mesh, which leads to benefit of efficiency and ease of output. The three type of regularization provided by PyMem3DG includes constraints on edge length, local triangle area, and local corner angles, adjusted by $K_{se}$, $K_{sl}$ and $K_{st}$, respectively. 

In [2]:
outputDir="output/tutorial2/ico_reg"
g = dg.System(icoFace, icoVertex, p)
g.meshProcessor.meshRegularizer.Kst = 1e-7
g.meshProcessor.meshRegularizer.Ksl = 1e-5
g.meshProcessor.meshRegularizer.Kse = 1e-7
g.meshProcessor.meshRegularizer.readReferenceData(icoFace, icoVertex, 0)
g.initialize()
fe = dg.Euler(g, 0.1, 50000, 1000, 0, outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
# fe.integrate()


In [8]:
import pymem3dg.visual as dg_vis
dg_vis.animate(outputDir+"/traj.nc", meanCurvature = True)

With the above regularization, the mesh looks significantly nicer, with all triangles closed to equilateral.

Free free to adjust the values of each type of regularization and attain intuition on the behavior. 

As you might observe, there are several disadvantages using the method of regularization. The behavior of simulation will depend on the strength of regularization. Local angle penalty is less restrictive and will minimally affect the underlying physics, but the resolution on high curvature region becomes very coarse. We could combine it with penalty on local area and more strongly edge length, but its influence on physics and optimization can not be neglected. In summary, how to find good balance between restriction and flexibility is not obvious. 

## Vertex shift
In addition, instead of constantly applying regularization force, one could also regularize the mesh once in a while, by "centering" the vertices (*barycenter* to be exact). This can be done by toggling the vertex shift option as follows,

In [4]:
outputDir = "output/tutorial2/ico_shift"
g = dg.System(icoFace, icoVertex, p)
g.meshProcessor.meshRegularizer.Kst = 0
g.meshProcessor.meshRegularizer.Ksl = 0
g.meshProcessor.meshRegularizer.Kse = 0
g.meshProcessor.meshMutator.isShiftVertex = True
g.initialize()
fe = dg.Euler(g, 0.1, 50000, 1000, 0, outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
fe.processMeshPeriod = 1000
# fe.integrate()


False

which also lead to a good mesh

## Mesh mutation

To resolve the challenges of regularization, PyMem3DG supports adaptive mesh by mesh mutation, including edge flip, edge split and edge collapse. Because of challenges mentioned above, mesh mutation should always be turned on when running complex simulation with large deformation. 

Notice that mesh mutation will most likely increase the computational cost and size of output files. 

The computational cost involves the time to loop through elements and decide whether conduct mesh mutation, and the subsequent overheads needed for recomputation of cached quantities. Such operation happens in the frequency of data output, therefore increasing the number of data output could better prevent the deterioration of mesh, but will increase computation.

Because of the varying mesh sizes, instead of single trajectory file using high performance *NetCDF* file, output file consists of series of snapshots in $\texttt{.ply}$ format. The detail of output files and visualization will be covered in the other tutorial. 

In [7]:
outputDir = "output/tutorial2/ico_mut"
g = dg.System(icoFace, icoVertex, p)
g.meshProcessor.meshMutator.isShiftVertex = True
g.meshProcessor.meshMutator.flipNonDelaunay = True
g.meshProcessor.meshMutator.splitFat = True
g.meshProcessor.meshMutator.splitSkinnyDelaunay = True
g.meshProcessor.meshMutator.splitCurved = True
g.meshProcessor.meshMutator.minimumEdgeLength = 0.001
g.meshProcessor.meshMutator.curvTol = 0.005
g.meshProcessor.meshMutator.collapseSkinny = True
g.meshProcessor.meshMutator.collapseSmall = True
g.meshProcessor.meshMutator.collapseFlat = True
g.meshProcessor.meshMutator.targetFaceArea = 0.0003
g.meshProcessor.meshMutator.isSmoothenMesh = True
g.initialize()
fe = dg.Euler(g, 1, 50000, 1000, 0, outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
fe.processMeshPeriod = 100
fe.integrate()


False

$\texttt{meshMutator}$ is used to specify conditions for mesh mutation, which should be specified after the instantiation of $\texttt{System}$. For details, please refer to the documentation. Mesh remains well-conditioned and well-resoluted during the simulation.

## Additional notes: 
 
### Reference mesh when mutation
Similarly, in theory we could also support specifying additional reference mesh. However, it is only necessary if the current input mesh has large deviation in total surface area from the reference mesh. Since membrane is nearly unstretchable, normally total surface area remains and self-referencing is sufficient. At the time of writing the tutorial, PyMem3DG will throw run-time error when the topology of reference mesh does not agree with the input mesh.

### Regularization + mutation
PyMem3DG does not recommend using regularization in conjunction with mesh mutation because it is most likely unnecessary to do so (as we see in previous example, vertex shift somewhat fills the role of regularization) and the behavior is not fully tested.